In [ ]:
import numpy as np
import math

# (この関数は前回のものから変更なし)
def check_hit_with_thickness(p0, vec, scint_end):
    """
    ミューオンの軌跡 (p0, vec) が、厚みのあるシンチレータ (scint_end) にヒットするか判定する。
    p0: 直線の始点 (x0, y0, z0)
    vec: 直線の方向ベクトル (vx, vy, vz)
    scint_end: ターゲットシンチレータのパラメータ辞書
    """
    x0, y0, z0 = p0
    vx, vy, vz = vec
    
    cx, cy = scint_end['shift_x'], scint_end['shift_y']
    r_in, r_out = scint_end['r_inner'], scint_end['r_outer']
    z_bot, z_top = scint_end['z_bottom'], scint_end['z_top']

    
########################################################################   
    # 軌跡のパラメータ表現: P(t) = p0 + t * vec
    # --- 判定1: 上面または下面との交差 ---
    if abs(vz) > 1e-9:
        t_top = (z_top - z0) / vz
        if t_top > 0:
            x_top = x0 + t_top * vx
            y_top = y0 + t_top * vy
            r_sq_top = (x_top - cx)**2 + (y_top - cy)**2
            if r_in**2 <= r_sq_top <= r_out**2:
                return True

        t_bot = (z_bot - z0) / vz
        if t_bot > 0:
            x_bot = x0 + t_bot * vx
            y_bot = y0 + t_bot * vy
            r_sq_bot = (x_bot - cx)**2 + (y_bot - cy)**2
            if r_in**2 <= r_sq_bot <= r_out**2:
                return True



    # --- 判定2: 側面（円柱）との交差 ---
    dx, dy = x0 - cx, y0 - cy
    A = vx**2 + vy**2
    B = 2 * (vx * dx + vy * dy)
    
    # Aがほぼ0の場合(軌跡がZ軸に平行)、側面とは交差しない
    if A < 1e-9:
        return False
        
    C_out = dx**2 + dy**2 - r_out**2
    delta_out = B**2 - 4 * A * C_out
    
    if delta_out >= 0:
        sqrt_delta = np.sqrt(delta_out)
        t1 = (-B + sqrt_delta) / (2 * A)
        t2 = (-B - sqrt_delta) / (2 * A)
        
        for t in [t1, t2]:
            if t > 0:
                z_intersect = z0 + t * vz
                if z_bot <= z_intersect <= z_top:
                    return True

    C_in = dx**2 + dy**2 - r_in**2
    delta_in = B**2 - 4 * A * C_in

    if delta_in >= 0:
        sqrt_delta = np.sqrt(delta_in)
        t1 = (-B + sqrt_delta) / (2 * A)
        t2 = (-B - sqrt_delta) / (2 * A)
        
        for t in [t1, t2]:
            if t > 0:
                z_intersect = z0 + t * vz
                if z_bot <= z_intersect <= z_top:
                    return True

    return False

###########################################################################

# ★★★ 新しくなったミューオン生成関数 ★★★
def generate_muon_track_from_volume(scint_start):
    """
    シンチレータの「体積内」のランダムな点から、
    「全方向(4π)」に等方的にミューオンを発生させる。
    """
    # 1. 最初のシンチレータの体積内にランダムな点を生成
    # z座標は厚みの範囲で一様
    z0 = np.random.uniform(scint_start['z_bottom'], scint_start['z_top'])
    
    # x,y座標は円環内で面積が一様になるように生成
    r = np.sqrt(np.random.uniform(scint_start['r_inner']**2, scint_start['r_outer']**2))
    theta_pos = np.random.uniform(0, 2 * np.pi)
    
    x0 = r * np.cos(theta_pos) + scint_start['shift_x']
    y0 = r * np.sin(theta_pos)
    
    p0 = np.array([x0, y0, z0])

    # 2. ミューオンの方向を全方向(4π)に等方的に生成
    #    球の表面に一様に分布する点を生成する標準的な方法
    phi = np.random.uniform(0, 2 * np.pi)       # 方位角
    costheta = np.random.uniform(-1, 1)         # cos(天頂角)
    theta = np.arccos(costheta)                 # 天頂角
    sintheta = np.sqrt(1.0 - costheta**2)
    
    # 3. 方向ベクトルを計算
    vx = sintheta * np.cos(phi)
    vy = sintheta * np.sin(phi)
    vz = costheta
    
    vec = np.array([vx, vy, vz])
    
    return p0, vec, theta


# --- メインのシミュレーション ---
if __name__ == '__main__':
    # シンチレータのパラメータ設定 (単位: cm)
    thickness = 1.0

    # 最初のシンチレータ（ミューオン発生源）
    scint_start = {
        'r_inner': 10.0,
        'r_outer': 20.0,
        'shift_x': -15.0, # 左に配置
        'shift_y': 0.0,
        'z_top': 0.0 + thickness / 2.0,
        'z_bottom': 0.0 - thickness / 2.0
    }

    # 2番目のシンチレータ（ターゲット）
    # Z座標を同じにして、側面->側面のケースをシミュレート
    scint_end = {
        'r_inner': 10.0,
        'r_outer': 20.0,
        'shift_x': 15.0, # 右に配置
        'shift_y': 0.0,
        'z_top': 0.0 + thickness / 2.0,
        'z_bottom': 0.0 - thickness / 2.0
    }

    # シミュレーション設定
    N_trials = 1000000  # 試行回数 (ヒット率が低いので多めに設定)
    count_hit = 0
    hit_angles_rad = []

    print(f"シミュレーションを開始します。試行回数: {N_trials}")
    print("発生モデル: シンチレータ1の体積内から全方向に等方的に発生")
    print(f"シンチレータ1 (発生源): 中心(x,y,z)=({scint_start['shift_x']}, 0, 0), r=[{scint_start['r_inner']},{scint_start['r_outer']}]")
    print(f"シンチレータ2 (ターゲット): 中心(x,y,z)=({scint_end['shift_x']}, 0, 0), r=[{scint_end['r_inner']},{scint_end['r_outer']}]")


    for i in range(N_trials):
        # ★★★ 新しい生成関数を呼び出す ★★★
        p0, vec, theta = generate_muon_track_from_volume(scint_start)
        
        # 2. 2番目のシンチレータにヒットしたか判定
        if check_hit_with_thickness(p0, vec, scint_end):
            count_hit += 1
            hit_angles_rad.append(theta)
            
        if (i + 1) % (N_trials // 10) == 0:
            print(f"進捗: {(i+1)/N_trials*100:.0f}%完了")


    print("\nシミュレーションが完了しました。")
    print("-" * 30)
    print(f"ヒット数: {count_hit}")
    print(f"全試行回数: {N_trials}")
    if N_trials > 0:
        acceptance = count_hit / N_trials
        print(f"アクセプタンス (ヒット率): {acceptance:.8f}")

    if count_hit > 0:
        avg_angle_deg = np.degrees(np.mean(hit_angles_rad))
        print(f"ヒットしたミューオンの平均天頂角: {avg_angle_deg:.2f} 度")

In [ ]:
#解読ver
import numpy as np
import math

dx = 1.0
dy = 1.0
dz = 1.0
dx2 = 1.0
dy2 = 1.0
dz2 = 1.0


#乱数計算の関数定義
def random_mc(label, r_inner_int, r_outer_int, shiftx_int, r_inner_end, r_outer_end, shiftx_end, zint, zend):

    # int上にランダムに点を生成
    r = np.sqrt(np.random.uniform(r_inner_int**2, r_outer_int**2))
    theta_r = np.random.uniform(0, 2*np.pi)
    x0 = r * np.cos(theta_r) + shiftx_int
    y0 = r * np.sin(theta_r)
    z0 = np.random.uniform(zint-1, zint)

    

    #上面との交差確認
    d_top = (zend - z0) / dz
    xa = x0 + d_top * dx
    ya = y0 + d_top * dy
    za = zend
    if r_inner_end**2 <= (xa-shiftx_end)**2 + (ya)**2 <= r_outer_end**2:
        count_hit[label] += 1
        angles_theta[label].append(np.degrees(theta))
        if len(lines[label]) < 100:
            lines[label].append(((x0, y0, z0), (xa, ya, zend)))
        return True

    #elseで次へ
    #下面との交差確認
    d_bot = (zend-1 - z0) / dz
    xa= x0 + d_bot * dx
    ya = y0 + d_bot * dy
    za = zend -1
    if r_inner_end**2 <= (xa-shiftx_end)**2 + (ya)**2 <= r_outer_end**2:
        count_hit[label] += 1
        angles_theta[label].append(np.degrees(theta))
        if len(lines[label]) < 100:
            lines[label].append(((x0, y0, z0), (xa, ya, zend)))
        return True


    #elseで次へ
    #側面との交差確認
    #At^2 + Bt + C = 0という二次方程式を解く

    A = dx**2 + dy**2
    B = 2 * ((x0-shiftx_end)*dx + y0*dy)

    if A < 1e-9: #傾きが小さすぎるとき、これないとバグる？
        return False
        
    #外側の側面との交差確認
    C_out = (x0-shiftx_end)**2 + y0**2 - r_outer_end**2
    delta_out = B**2 - 4*A*C_out
    if delta_out >= 0:
        sqrt_delta = np.sqrt(delta_out)
        t1 = (-B + sqrt_delta) / (2 * A)
        t2 = (-B - sqrt_delta) / (2 * A)
        for t in [t1, t2]:
            if t > 0:
                z_intersect = z0 + t*dz
                if zend-1 <= z_intersect <= zend:
                    count_hit[label] += 1
                    angles_theta[label].append(np.degrees(theta))
                    if len(lines[label]) < 100:
                        xa = x0 + t * dx
                        ya = y0 + t * dy
                        lines[label].append(((x0, y0, z0), (xa, ya, zend)))
                    return True

    #内側の側面との交差確認
    C_in = (x0-shiftx_end)**2 + y0**2 - r_inner_end**2
    delta_in = B**2 - 4*A*C_in
    if delta_in >= 0:
        sqrt_delta = np.sqrt(delta_in)
        t1 = (-B + sqrt_delta) / (2 * A)
        t2 = (-B - sqrt_delta) / (2 * A)
        for t in [t1, t2]:
            if t > 0:
                z_intersect = z0 + t * dz
                if zend-1 <= z_intersect <= zend:
                    count_hit[label] += 1
                    angles_theta[label].append(np.degrees(theta))
                    if len(lines[label]) < 100:
                        xa = x0 + t * dx
                        ya = y0 + t * dy
                        lines[label].append(((x0, y0, z0), (xa, ya, zend)))
                    return True

    return False
    #どこかでTrueだったら、count_hit, angles_theta, lines保存


    # # endの中心からの距離で判定
    # if r_inner_end**2 <= (xa-shiftx_end)**2 + (ya)**2 <= r_outer_end**2:
    #     count_hit[label] += 1
    #     angles_theta[label].append(np.degrees(theta))
    #     if len(lines[label]) < 100:
    #         lines[label].append(((x0, y0, zint), (xa, ya, zend)))
    # if r_inner_end**2 <= (xb-shiftx_end)**2 + (yb)**2 <= r_outer_end**2:
    #     count_hit2[label] += 1
    #     angles_theta2[label].append(np.degrees(theta2))